# Колокації

In [0]:
import warnings
import stanfordnlp
from typing import List, Dict

In [0]:
warnings.filterwarnings('ignore')

In [5]:
stanfordnlp.download('en')

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)
Y

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /root/stanfordnlp_resources/en_ewt_models.zip


100%|██████████| 235M/235M [00:03<00:00, 64.2MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.


In [0]:
stanfordnlp.download('uk')

### 1. Англійська мова

У цьому завданні вам потрібно дослідити сполучуваність дієслів одного синонімного ряду з прислівниками. Наприклад, ми частіше кажемо "love somebody dearly", "honor somebody highly", але "admire somebody greatly".

#### Завдання:

#### 1. продовжте синонімний ряд дієслів: "say", "tell", "speak", "claim", "communicate"

"say", "tell", "speak", "claim", "communicate", "express", "talk", "report", "notify", "argue", "report"

In [0]:
verbs = ["say", "tell", "speak", "claim", "communicate", 
         "express", "talk", "report", "notify", "argue", "report"]

#### 2. напишіть функцію, яка знаходить у реченні дієслово (за складеним раніше синонімним рядом) і витягає усі можливі прислівники на "-ly", якими це дієслово керує

In [7]:
nlp = stanfordnlp.Pipeline()

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand

##### Необхідна функція представленна нище

In [0]:
def find_verbs_with_associated_adverbs(nlp, text: str, verbs: List[str], 
                                       verb_adverb: Dict[str, Dict[str, int]] = None):
    doc = nlp(text)
    
    if verb_adverb is None:
        verb_adverb = dict()
    
    for sent in doc.sentences:
        for token in sent.dependencies:
            if token[0].lemma not in verbs:
                continue

            if token[2].upos !='ADV':
                continue


            verb = token[0].lemma
            adverb = token[2].lemma
            
            if adverb[-2:] != 'ly':
                continue

            if verb not in verb_adverb:
                verb_adverb[verb] = dict()

            if adverb not in verb_adverb[verb]:
                verb_adverb[verb][adverb] = 1
            else:
                verb_adverb[verb][adverb] += 1        
    
    return verb_adverb

##### Приклад використання даної функції

In [0]:
sentence_example = "I was curious how the Obama campaign was going to deal with this new development , since they had claimed loudly last week that al-Maliki had definitively moved away from McCain 's position and over to their own.\n"

In [10]:
find_verbs_with_associated_adverbs(nlp, text=sentence_example, verbs=verbs)

{'claim': {'loudly': 1}}

#### 3. напишіть програму, яка знайде усі можливі прислівники для ваших дієслів у корпусі блогів

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [20]:
!ls 'gdrive/My Drive/NLP/Task-1/data'

blog2008.txt


In [0]:
corpus_file = "gdrive/My Drive/NLP/Task-1/data/blog2008.txt"

In [0]:
def read_file(file: str):
    lines = []
    with open(file) as file_in:        
        for line in file_in:
            lines.append(line)
            
    return lines

In [0]:
corpus = read_file(corpus_file)

In [0]:
def find_all_adverb_with_associated_verbs(nlp, corpus: List[str], verbs: List[str]):
    result = dict()
    for text in corpus:
        find_verbs_with_associated_adverbs(nlp, text=text, verbs=verbs, verb_adverb=result)
    
    return result
        

In [24]:
%time verb_assoc_adverb = find_all_adverb_with_associated_verbs(nlp, corpus[:100], verbs)

CPU times: user 5.29 s, sys: 38.5 ms, total: 5.33 s
Wall time: 5.33 s


In [26]:
(len(corpus) / 100) * 5.33 / 3600

4.500800055555556

In [27]:
verb_assoc_adverb

{'claim': {'initially': 1}}

#### 4. на виході програма повинна видати десять найчастотніших прислівників для кожного дієслова (разом із частотою). Приклад виводу:


say: (loudly, 51), (silently, 45), (quietly, 10)<br/>
tell: (quietly, 100), (loudly, 61), (seriously, 5)

In [0]:
def analyze_adverb_with_associated_verbs(nlp, corpus, verbs):
    verb_assoc_adverb = find_all_adverb_with_associated_verbs(nlp, corpus, verbs)
    
    for verb in verb_assoc_adverb:
        print(f"{verb}:", end=' ')
        for adverb in verb_assoc_adverb[verb]:
            count = verb_assoc_adverb[verb][adverb]
            print(f"({adverb}, {count}), ", end='')
            
        print('')

#### 5. збережіть програму та результати аналізу на корпусі (вивід) у директорії з вашим іменем


In [29]:
%time analyze_adverb_with_associated_verbs(nlp, corpus[:10], verbs)

CPU times: user 521 ms, sys: 6.15 ms, total: 527 ms
Wall time: 532 ms


Remark: The results below contains also adverbs without '-ly', becuase firstly this was forgotten to consider and was run without consideration 

In [32]:
analyze_adverb_with_associated_verbs(nlp, corpus, verbs)

claim: (initially, 3), (also, 61), (still, 11), (however, 14), (where, 9), (brazenly, 1), (far, 3), (so, 12), (surely, 2), (down, 5), (just, 10), (solemnly, 1), (previously, 8), (first, 10), (now, 64), (falsely, 78), (perhaps, 1), (oddly, 1), (already, 2), (meanwhile, 5), (apparently, 2), (never, 8), (when, 47), (abroad, 1), (then, 34), (only, 6), (ahead, 3), (fiercely, 1), (how, 10), (yet, 4), (again, 6), (alternatively, 1), (of, 3), (instead, 2), (even, 15), (constantly, 1), (loudly, 1), (annually, 1), (bizarrely, 2), (outright, 2), (earlier, 7), (actually, 3), (piously, 1), (ago, 8), (really, 2), (recently, 4), (preposterously, 1), (notably, 1), (widely, 1), (hardly, 1), (over, 4), (absurdly, 3), (ever, 2), (furthermore, 1), (clearly, 2), (why, 4), (probably, 1), (finally, 1), (namely, 1), (illegally, 1), (righteously, 1), (otherwise, 5), (indignantly, 2), (long, 2), (not, 4), (immediately, 2), (explicitly, 1), (tearfully, 1), (somehow, 1), (patronizingly, 1), (once, 6), (indeed, 1)

### 2. Українська мова

In [0]:
nlp_uk = stanfordnlp.Pipeline(lang="uk")

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/dmytro.babenko/stanfordnlp_resources/uk_iu_models/uk_iu_tokenizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/dmytro.babenko/stanfordnlp_resources/uk_iu_models/uk_iu_tagger.pt', 'pretrain_path': '/Users/dmytro.babenko/stanfordnlp_resources/uk_iu_models/uk_iu.pretrain.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/dmytro.babenko/stanfordnlp_resources/uk_iu_models/uk_iu_lemmatizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/Users/dmytro.babenko/stanfordnlp_resources/uk_iu_models/uk_iu_parser.pt', 'pretrain_path': '/Users/dmytro.babenk

#### 1. напишіть функцію, яка знаходить у реченні іменник, що позначає істоту, і витягає усі прикметники, якими цей іменник керує


In [0]:
def is_token_anim(token):
    if 'Animacy=Anim' in token.feats:
        return True
    return False

In [0]:
def find_noun_with_associated_adj(nlp, text: str, noun_adj_dict : Dict[str, Dict[str, int]] = None):
    doc = nlp(text)
    
    if noun_adj_dict is None:
        noun_adj_dict = dict()
    
    for sent in doc.sentences:
        for token in sent.dependencies:   
                        
            if token[0].upos != 'ADJ' and token[0].upos != 'NOUN':
                continue

            if token[2].upos != 'ADJ' and token[2].upos != 'NOUN':
                continue
                
            
            if token[0].upos == 'ADJ' and token[2].upos != 'NOUN':
                continue
                
            if token[0].upos == 'NOUN' and token[2].upos != 'ADJ':
                continue
                     
            noun_token = token[0] if token[0].upos == 'NOUN' else token[2]
            
            #перевірка чи іменнки позначає істоту
            if is_token_anim(noun_token) is False:
                continue
            
            adj_token = token[0] if token[0].upos == 'ADJ' else token[2]
            
            noun = noun_token.text
            adjective = adj_token.text
            
            if noun not in noun_adj_dict:
                noun_adj_dict[noun] = { adjective : 1}
            else:
                if adjective not in noun_adj_dict[noun]:
                    noun_adj_dict[noun][adjective] = 1
                else:
                    noun_adj_dict[noun][adjective] += 1
                            
    return noun_adj_dict        

In [0]:
sentence_example = "Чоловік був щирий та добре поводився зі своїм котом."

In [0]:
find_noun_with_associated_adj(nlp_uk, sentence_example)

{'Чоловік': {'щирий': 1}}

#### 2. напишіть програму, яка знайде усі можливі словосполучення прикметників та іменників-істот у романі "Тигролови"

In [0]:
tyhrolovy_file = "../../../tasks/02-structural-linguistics/data/tyhrolovy.txt"

In [0]:
tyhrolovy_corpus = read_file(tyhrolovy_file)

In [0]:
def find_noun_with_associated_adj_for_corpus(nlp, corpus: List[str]):
    result = dict()
    for text in corpus:
        if text is None or len(text) <= 1:
            continue 
        find_noun_with_associated_adj(nlp, text, result)
    return result

In [0]:
noun_adj_dict = find_noun_with_associated_adj_for_corpus(nlp_uk, tyhrolovy_corpus[:50])

In [0]:
noun_adj_dict

{'лось': {'сохатий': 1},
 'дракон': {'легендарний': 1, 'реальний': 2, 'справжній': 1, 'найбільший': 1},
 'драконів': {'найстрашніший': 1},
 'голова': {'вогненноока': 1},
 'циклоп': {'велетенський': 1, 'двоокий': 1},
 'мертвяків': {'живих': 1},
 'дракона': {'натоптане': 1}}

#### 3. на виході програма повинна видати усі знайдені словосполучення та їх частоти. Приклад виводу: <br/> 
84: старий сірко <br/> 
5: великий начальник <br/> 

In [0]:
def analyze_noun_with_associated_adjectives(nlp, corpus):
    noun_adj_dict = find_noun_with_associated_adj_for_corpus(nlp, corpus)
    for noun in noun_adj_dict:
        for adjective in noun_adj_dict[noun]:
            phrase = f"{adjective} {noun}"
            count = noun_adj_dict[noun][adjective]

            print(f"{count}: {phrase}")

#### 4. збережіть програму та результати виводу в директорії з вашим іменем

In [0]:
analyze_noun_with_associated_adjectives(nlp_uk, tyhrolovy_corpus)

1: сохатий лось
1: легендарний дракон
2: реальний дракон
1: справжній дракон
1: найбільший дракон
1: найстрашніший драконів
1: вогненноока голова
1: примружені голова
1: непокрита голова
1: оленяча голова
1: мокра голова
1: Другий голова
1: велетенський циклоп
1: двоокий циклоп
1: живих мертвяків
1: натоптане дракона
1: великий начальник
1: сердитий начальник
1: розцяцькований начальник
1: більший начальник
1: Великий начальник
1: дурний начальник
2: рідний товаришу
1: надзвичайне власника
1: обірваних людей
1: випадкових людей
1: змарнованих людей
1: списаних людей
1: молодих людей
1: різних людей
1: простих людей
1: змучених людей
1: чудодійної людей
1: силенну людей
1: напівголих людей
1: обірваних пращури
1: худих скелети
1: зігнані вівці
1: оточені вівці
1: божевільний маніяк
1: безумний сміливцю
1: гордого сокола
1: безумного сміливця
1: гордого нащадка
1: першого каторжанина
1: страшного злочинця
1: державного злочинця
1: русявий Юнак
1: експансивними мешканцями
1: заселені мешк